In [9]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pymc3 as pm
from pymc3.ode import DifferentialEquation
print('Running on PyMC3 v{}'.format(pm.__version__))

from bs4 import BeautifulSoup
import requests
import re

import os

from sklearn.externals import joblib 

import pandas as pd

Running on PyMC3 v3.8


In [2]:
country = 'Canada'
file = os.path.join('csv_out',country+'.csv')
country_df = pd.read_csv(file)
country_df

def get_country_sir(country, start_date='', end_date='', min_cases=10):
    
    # update for any country
    # italy
    population = 60e6
    
    country = country.title().replace(' ', '_')
    file = os.path.join('csv_out',country+'.csv')
    country_df = pd.read_csv(file)
    
    start = country_df[country_df.Date == start_date].index
    if len(start)==0:
        start = 0
    else:
        start = start[0]
        
    end = country_df[country_df.Date == end_date].index
    if len(end)==0:
        end = country_df.index[-1]
    else:
        end = end[0]
        
    dates = country_df.loc[start:end + 1, 'Date'].values
    data = country_df.loc[start:end + 1, 'Confirmed'].values
    deaths = country_df.loc[start:end + 1, 'Deaths'].values
    recovered = country_df.loc[start:end+1, 'Recovered'].values

    if max(data) < min_cases:
        print('Warning, {:d} cases has not occured in this date range.')
    else:
        min_start = np.where(np.array(data) >= min_cases)[0][0]
        data = data[min_start:]
        dates = dates[min_start:]
        deaths = deaths[min_start:]
        recovered = recovered[min_start:]
        
    # infected = total cases - deaths - recoveries
    infected = data - deaths - recovered
    
    # susceptible = population - infected - deaths - recovered
    susceptible = population - infected - deaths - recovered
        
    

    return dates, np.arange(1, len(data) + 1), susceptible/population, infected/population
    
    

In [23]:
dates, x, sus, inf = get_country_sir('italy', min_cases=100)
# sus and inf are already normalized
# just normalize x
x_train = x[:-3]
x_test = x[-3:]

scalex = MinMaxScaler()
x_scale = scalex.fit_transform(x_train.reshape(-1,1)).flatten()

sus_train = sus[:-3]
sus_test = sus[-3:]

inf_train = inf[:-3]
inf_test = inf[-3:]

# make single array
y_train = np.hstack((sus_train.reshape(-1,1), inf_train.reshape(-1,1)))
y_test = np.hstack((sus_test.reshape(-1,1), inf_test.reshape(-1,1)))

y_train.shape

y0 = [y_train[0][0], y_train[0][1]]

In [29]:
def sir_function(y, t, p):
    # 'constants'
    delta = p[0]
    lmbda = p[1]
    beta = p[2]*pm.math.exp(-t*delta)
    
    # y = (s, i)
    
    # susceptible differential
    ds = -y[0]*y[1]*beta
    
    # infected differential
    di = y[0]*y[1]*beta - y[1]*lmbda
    
    return [ds, di]


sir_model = DifferentialEquation(
    func=sir_function,
    times=x_scale,
    n_states=2, # number of y (sus and inf)
    n_theta=3, # number of parameters (lambda, delta, beta_0)
    t0=0
)

In [ ]:
with pm.Model() as model4:
    
    print('Initializing Priors')
    sigma = pm.HalfCauchy('sigma', 1, shape=2)

    # R0 is bounded below by 1 because we see an epidemic has occured
    R0 = pm.Bound(pm.Normal, lower=1)('R0', 2,3)
    lmbda = pm.Lognormal('lambda',pm.math.log(2), 2)
    delta = pm.Lognormal('delta', pm.math.log(2), 2)
    beta = pm.Deterministic('beta', lmbda*R0)
    
    
    print('Setting up model')
    sir_curves = sir_model(y0=y0, theta=[lmbda, delta, beta])

    Y = pm.Lognormal('Y', mu=pm.math.log(sir_curves), sd=sigma, observed=y_train)

    print('Starting sampling')
    #prior = pm.sample_prior_predictive(progressbar=True)
    trace = pm.sample(2000,tune=1000, target_accept=0.9, cores=4, progressbar=True)
    posterior_predictive = pm.sample_posterior_predictive(trace, progressbar=True)

    #data = az.from_pymc3(trace=trace, prior = prior, posterior_predictive = posterior_predictive)

Initializing Priors
Setting up model


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Starting sampling


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [delta, lambda, R0, sigma]
